# Imports

In [1]:
from ultralytics import YOLO
import os
from scipy.io import loadmat
import numpy as np
import shutil
import math
import scipy.io

In [46]:
# load model YOLO v8
model = YOLO("yolov8m-pose.pt")

# configurations
N = 10
MPII_IMAGES_DIR = "./mpii/images"
IMAGE_FILES = [os.path.join(MPII_IMAGES_DIR, file) for file in os.listdir(MPII_IMAGES_DIR) if file.endswith(('.jpg'))][:N]
yolo_images_dir = "./YOLO_images"

count = 0
results_list = []


In [59]:
# USE ANNOTATION MPII
annotations_data = loadmat("./mpii/annotations/mpii_human_pose.mat")
annolist = annotations_data['RELEASE'][0][0]['annolist'] 
#print("annotations_data", annotations_data)
#annolist_test = annotations_data['RELEASE'] ['annolist']


#print("Data Structure annolist test", annolist_test)
#name_images= annolist_test[0]['image']['name']
#print(name_images)
# annolist_test  tridimensional matrix
##############################################################
#for entry in annolist:
    #image_names = entry['image'][:10]
    #print("IMAGE NAMES ANN", image_names)
#######################################################################
# Iterar sobre as primeiras 10 entradas da lista de anotações
#for entry in annolist[:10]:
    # Extrair todos os dados da entrada
    #image_data = entry['image'][:10]
    #print("IMAGE DATA", image_data)
    # Como cada entrada pode conter múltiplas imagens (caso de múltiplas anotações para a mesma imagem),
    # vamos iterar sobre essas imagens
    #for img in image_data:
        #image_name = img['name'][0][0]  # Obtendo o nome da imagem
        #print("Image NAME:", image_name)
#############################################################################
# Iterar sobre as primeiras 10 entradas da lista de anotações
for entry in annolist[:10]:
    # Extrair todos os dados da entrada
    image_data = entry['image'][:10]
    #print("IMAGE DATA", image_data)

    # each entry can contain multiple images (case of multiple annotations for the same image). iterate over these images
    for img_info in image_data:
        image_name = img_info['name'][0][0] 
        image_name = image_name.item()  # Convert to string
        print("IMAGE_NAME", image_name)
        image_path = os.path.join(MPII_IMAGES_DIR, image_name)
        print("IMAGE PATH", image_path)


IMAGE_NAME 037454012.jpg
IMAGE PATH ./mpii/images\037454012.jpg
IMAGE_NAME 095071431.jpg
IMAGE PATH ./mpii/images\095071431.jpg
IMAGE_NAME 073199394.jpg
IMAGE PATH ./mpii/images\073199394.jpg
IMAGE_NAME 059865848.jpg
IMAGE PATH ./mpii/images\059865848.jpg
IMAGE_NAME 015601864.jpg
IMAGE PATH ./mpii/images\015601864.jpg
IMAGE_NAME 015599452.jpg
IMAGE PATH ./mpii/images\015599452.jpg
IMAGE_NAME 005808361.jpg
IMAGE PATH ./mpii/images\005808361.jpg
IMAGE_NAME 086617615.jpg
IMAGE PATH ./mpii/images\086617615.jpg
IMAGE_NAME 060111501.jpg
IMAGE PATH ./mpii/images\060111501.jpg
IMAGE_NAME 070807258.jpg
IMAGE PATH ./mpii/images\070807258.jpg


In [3]:
for file in os.listdir(MPII_IMAGES_DIR):
    if file.endswith('.jpg'):
        image_file = os.path.join(MPII_IMAGES_DIR, file)
        shutil.copy(image_file, yolo_images_dir)
        
        # execute detection for YOLO_images directory
        results = model(source=image_file, show=False, conf=0.3, save=False)
        results_list.append(results)
        #print("RESULTS LIST", results_list)
        count += 1
        if count >= N:
            break


image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000111209.jpg: 480x640 1 person, 854.7ms
Speed: 0.0ms preprocess, 854.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000114595.jpg: 384x640 2 persons, 710.9ms
Speed: 2.3ms preprocess, 710.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000120361.jpg: 480x640 11 persons, 743.0ms
Speed: 1.6ms preprocess, 743.0ms inference, 39.6ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000124344.jpg: 384x640 10 persons, 716.6ms
Speed: 0.0ms preprocess, 716.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\bp111\

In [3]:
# acess and print predictid keypoints MODEL YOLO
for i,result in enumerate(results_list):
    for r in result:
        #print(len(result))
        image_name = r.path.split('\\')[-1]
        #print(f"image {i+1} ({image_name})")
        xyn = r.keypoints.xyn
        for j, pose in enumerate(xyn):
            #print(f"pose {j+1}")
            for k, landmark in enumerate(pose):
                x, y = landmark
                #print(f"keypoints {k+1}:\t ({x},\t {y})") 

